<a href="https://colab.research.google.com/github/shryu8902/KIRD_AUTOML/blob/main/KIRD_Day_1_2_ML_%EB%AA%A8%EB%8D%B8_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 머신러닝 모델 기본 파이프라인
데이터 준비 - 전처리 - 모델준비 - 학습 - 평가 순서로 구성된 가장 기본적인 머신러닝 모델 학습 파이프라인에 대한 실습을 진행해보도록 하겠습니다.

이번 실습에서는 타이타닉 생존자 데이터셋으로, 생존 여부를 예측하는 랜덤포레스트 모델을 학습할 것입니다.


## 0. 문제 정의
머신러닝 모델 개발에 앞서 문제에 대한 정의가 선행되어야 합니다.
본 예제에서 문제는 생존 여부에 대해 예측하는 문제이므로, 분류 문제에 해당합니다.
또한 다루고자 하는 타이타닉 데이터셋은 타이타닉호 탑승자의 성별, 나이, 운임 정보 등을 포함하고 있습니다.

## 1. 데이터 준비
데이터 다운로드 및 필요 라이브러리 준비

In [1]:
# 기초 라이브러리 준비
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# 예제 데이터인 타이타닉 데이터셋을 다운로드 하고 pandas dataframe으로 불러오겠습니다.

!wget https://raw.githubusercontent.com/shryu8902/KIRD_AUTOML/main/Titanic-Dataset.csv

titanic_data = pd.read_csv('./Titanic-Dataset.csv')

--2024-11-19 15:16:16--  https://raw.githubusercontent.com/shryu8902/KIRD_AUTOML/main/Titanic-Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘Titanic-Dataset.csv’

Titanic-Dataset.csv 100%[===================>]  59.76K  --.-KB/s    in 0.01s   

2024-11-19 15:16:17 (4.54 MB/s) - ‘Titanic-Dataset.csv’ saved [61194/61194]



### 데이터 살펴보기
데이터 전처리에 들어가기 전, 데이터가 어떤식으로 구성되어있는지 간단한 시각화 등을 통해서 살펴보도록하겠습니다.

데이터셋은 다음의 데이터를 포함하고있습니다.
- PassengerId: 탑승자 인덱스

- Survived: 탑승자가 생존했는지 여부를 나타냅니다. 0은 생존하지 못했음을, 1은 생존했음을 의미합니다.

- Pclass: 티켓의 클래스로, 1은 1등실(상류층), 2는 2등실(중류층), 3은 3등실(하류층)을 의미.

- Name: 탑승자 성명.

- Sex: 탑승자의 성별.

- Age: 탑승자의 나이. 일부 미성년자의 경우 "xx.5"의 형태로 소수점이 있을 수 있습니다.

- SibSp: 함께 탑승한 형제자매나 배우자의 수.

- Parch: 함께 탑승한 부모님 혹은 자녀의 수.

- Ticket: 티켓 번호.

- Fare: 탑승자가 지불한 운임.

- Cabin: 탑승자의 객실 번호.

- Embarked: 탑승자가 승선한 항구. C는 Cherbourg, Q는 Queenstown, S는 Southampton을 의미.

In [3]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
titanic_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
titanic_data.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


위와 같은 분석 내용을 바탕으로 다음의 내용에 대한 전처리를 수행해보도록하겠습니다.

1. Cabin의 경우 NaN 값이 대부분이므로 해당 컬럼 제외.
2. Age 또한 NaN 값이 전체의 약 20%를 차지하므로 누락값에 대한 처리가 필요함. 나이가  생존 여부에 중요한 영향을 미칠 것으로 생각되므로 누락값을 지칭하는 특정값을 처리 (-1)
3. 이름, 티켓번호, 탑승항구, passenger ID는 생존여부와 상관성이 떨어질 것으로 예상되므로 컬럼 제외.
4. 성별이 'male', 'female'의 텍스트로 저장되어있으므로, 0,1로 변경.
    -> scikit learn LabelEncoder 활용 (범주형 변수를 수치형 변수로 변환)


In [7]:
# 데이터 전처리
from sklearn.preprocessing import LabelEncoder

# 1, 3, 특정 컬럼 제외
# 컬럼을 제외하기 위해서 axis=1 설정.
titanic_data.drop(['PassengerId','Cabin','Name','Ticket','Embarked'],axis = 1, inplace=True)

# 2, Age NaN값 처리
titanic_data.Age.fillna(-1, inplace = True)

# 4, 범주형 변수 -> 수치형 변수 변환 (e.g., [A,B,C] -> [0, 1, 2])
titanic_data['Sex'] = LabelEncoder().fit_transform(titanic_data['Sex'])  # 'sex' feature를 numerical로 변환

<ipython-input-7-2acf512fbf52>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data.Age.fillna(-1, inplace = True)


In [8]:
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [36]:
# 데이터셋 준비
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0) # Stratified split 적용


## 모델 준비, 학습, 그리고 평가

In [10]:
# 모델 준비 및 학습
random_forest = RandomForestClassifier(n_estimators=5, random_state = 0)
random_forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [11]:
# 모델 평가
y_pred = random_forest.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.776536312849162


## 성능의 변화

In [12]:
# 모델 Hyperparameter에 의한 변화
RF1 = RandomForestClassifier(n_estimators = 50, max_depth = 5, random_state = 0)
RF1.fit(X_train, y_train)
print("Accuracy: ", accuracy_score(y_test, RF1.predict(X_test)))

Accuracy:  0.7988826815642458


In [13]:
# 입력 데이터 구성에 의한 변화
RF2 = RandomForestClassifier(n_estimators = 50, max_depth = 5, random_state = 0)
RF2.fit(X_train.drop('Age',axis=1), y_train)
print("Accuracy: ", accuracy_score(y_test, RF2.predict(X_test.drop('Age',axis=1))))

Accuracy:  0.7541899441340782


In [15]:
# 모델에 따른 변화
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state = 0)
DT.fit(X_train, y_train)
print("Accuracy: ", accuracy_score(y_test, DT.predict(X_test)))

Accuracy:  0.7486033519553073


모델 학습으로 완료되는 것이 아님.
모델의 성능은 다양한 요인에 의해 바뀔 수 있으며,
크게 세가지로 구분할 수 있습니다.

1. 모델 자체의 변화.

2. 모델 하이퍼 파라미터의 변화.

3. 입력 데이터 구성의 변화.

> 즉 하나의 인공지능 모델 = 한 명의 학생이라고 가정했을 때,
> 우리는 각 반을 대표하는 최고의 학생을 뽑는 것이 목표.
>> 자체적으로 대표를 뽑기위해서는? 내부적으로 테스트가 필요 (validation set에 대한 성능)

이러한 모델 변화에 대한 실험을 가장 단순한 반복문으로 구성하면 다음과 같습니다.

```python
for train, val, test in K-folds
    for model in models:
        for params in hyper_params:
            for feature in feature_combination:
                training model
                get results on validation set
                evaluate validation results
```

결과적으로 이러한 모델 관련 요소의 변화에 대한 실험을 쉽게 할 수 있도록 지원하는 것이 Auto ML 라이브러리 할 수 있습니다.

1. 모델 자체의 변화 -> 다양한 모델에 대한 시도 -> AutoML 라이브러리
2. 모델 하이퍼 파라미터의 변화 -> grid search, random search, bayesian optimization 혹은 AutoML 라이브러리
3. 입력 데이터 구성의 변화 -> Feature engineering 라이브러리 혹은 AutoML 라이브러리





# 2. 하이퍼 파라미터 최적화

AutoML 라이브러리를 활용하여 자동적으로 하이퍼 파라미터 최적화를 수행하기 전에,
각 하이퍼 파라미터 최적화 방법론에 대해서 개별적으로 실습해 보도록 하겠습니다.

## 1. Grid search
그리드 서치는 사전에 탐색해야 할 하이퍼파라미터 공간 및 구간을 설정해두고,
모든 하이퍼파라미터 변수 조합에 대한 테스트를 수행, 최적의 조합을 찾는 방법입니다.

- 장점 : 직관적, 단순함.
- 단점 : 탐색해야 할 구간이 증가할 수록 계산 자원, 시간 등이 많이 소요됨.


**따라서 탐색해야할 하이퍼 파라미터가 상대적으로 적고 탐색범위가 명확할 때 활용도가 높음**

그리드 서치는 scikit-learn library의
```GridSearchCV()``` 클래스를 통해 수행 할 수 있으며, 해당 클래스의 주요 파라미터는 다음과 같습니다.

- ```estimator```: scikit learn 인터페이스를 따르는 훈련시킬 모델을 설정.

- ```param_grid```: 탐색 할 하이퍼파라미터 조합에 대한 딕셔너리 또는 딕셔너리들의 리스트. 키는 하이퍼파라미터 변수명, 이에 대응하는 밸류는 하이퍼파라미터 값에 대응.

- ```cv```: K-fold CV를 위한 K 값. 또는 Cross validation generator

- ```refit```: True 설정시, 가장 좋은 하이퍼파라미터를 찾은 후 전체 데이터셋에 대해 다시 훈련시킵니다. ```best_estimator_``` 활용 가능.





In [16]:
from sklearn.model_selection import GridSearchCV

# 기본 모델 설정
RF = RandomForestClassifier(random_state=0)

# 탐색하기 위한 하이퍼파라미터 설정
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 1, 5],
    'min_samples_split': [2, 3]
}

# 전체 그리드 서치 경우의 수
# 2*3*2 = 12
# 전체 그리드 서치 실험 횟수
# 12*3 (CV=3) = 36

# 그리드 서치 인스턴스 설정 및 수행
# 3개 fold로 구분하여 hyperparameter 탐색
grid_search = GridSearchCV(estimator=RF, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters: ", grid_search.best_params_)

# 최적의 하이퍼파라미터로 학습된 모델 출력
print("Best model: ",grid_search.best_estimator_)

#

Best parameters:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best model:  RandomForestClassifier(random_state=0)


In [17]:
print("Accuracy: ", accuracy_score(y_test, grid_search.predict(X_test)))

Accuracy:  0.7932960893854749


In [18]:
# 그리드 서치의 결과는 딕셔너리 형태로 주어지기때문에 접근 가능한 키를 살펴봅니다.
print(grid_search.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_samples_split', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [19]:
# 그리드 서치 내 각 파라미터 조합 시도에 대한 성능을 살펴볼 수 있습니다.
print(grid_search.cv_results_["mean_test_score"])
print(grid_search.cv_results_["rank_test_score"])

[0.81311681 0.81312863 0.81172216 0.81312272 0.77245919 0.77386566
 0.77245919 0.77386566 0.81035705 0.81036887 0.81177534 0.81176943]
[ 3  1  6  2 11  9 11  9  8  7  4  5]


In [ ]:
# 랭크 기준으로 [1] 인덱스의 결과의 파라미터와 best_params_ 어트리뷰트로 얻은 파라미터를 비교합니다.
print(grid_search.cv_results_['params'][1])
print(grid_search.best_params_)

{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


## Random search

랜덤 서치는 그리드 서치와 비슷하지만, 모든 가능한 하이퍼파라미터 조합을 탐색하는 대신 주어진 분포에서 하이퍼파라미터를 무작위로 선택하여 테스트하는 방법입니다.

- 장점: 그리드 서치에 비해 계산 자원을 적게 사용하면서 최적의 결과를 찾을 가능성이 높습니다.
- 단점: 완전한 탐색을 하지 않기 때문에 최적의 하이퍼파라미터를 놓칠 수 있습니다.

**따라서, 탐색해야할 하이퍼파라미터가 많거나 탐색 범위가 불확실한 경우에 적합합니다.**

랜덤 서치는 scikit-learn 라이브러리의 ```RandomizedSearchCV()``` 클래스를 사용하여 수행할 수 있습니다.

주요 입력 파라미터는 GridSearch와 유사하나 param_grid 대신 param_distribution과 n_iter가 필요합니다.
- ```param_distributions```: 탐색할 하이퍼파라미터의 **분포**를 설정합니다. 딕셔너리 형태로 하이퍼파라미터의 키에 대한 변수와 샘플링 할 분포가 필요합니다.
- ```n_iter```: 분포에서 하이퍼 파라미터 샘플링할 횟수를 설정합니다. 그리드 서치에서는 ```param_grid```에서 가능한 모든 조합에 대해 수행한다면, 랜덤 서치에서는 가능한 분포에서 ```n_iter```만큼의 파라미터 조합을 탐색합니다.

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# 기본 모델 설정
RF = RandomForestClassifier(random_state=0)

# 탐색하기 위한 파라미터 분포 설정
param_dist = {
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_depth": randint(3, 10),
    "n_estimators": randint(10, 100),
    "max_features": randint(1, 9),
}

# 랜덤 서치 인스턴스 설정 및 수행
random_search = RandomizedSearchCV(
    estimator=RF,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    random_state=0
)

# 랜덤 서치 실행
random_search.fit(X_train, y_train)

# 결과 출력
print("Best parameters: ", random_search.best_params_)
print("Best model: ", random_search.best_estimator_)


Best parameters:  {'criterion': 'entropy', 'max_depth': 6, 'max_features': 6, 'n_estimators': 46}
Best model:  RandomForestClassifier(criterion='entropy', max_depth=6, max_features=6,
                       n_estimators=46, random_state=0)


In [21]:
# 랜덤 서치로 얻은 best_estimator에 대한 평가
print("Accuracy: ", accuracy_score(y_test, random_search.predict(X_test)))

Accuracy:  0.8491620111731844


In [22]:
print(random_search.cv_results_["mean_test_score"])
print(random_search.cv_results_["rank_test_score"])

[0.81878405 0.82719923 0.77386566 0.80332471 0.80474299 0.81877814
 0.81740122 0.80333652 0.81736576 0.79349715 0.80752639 0.82019052
 0.81034524 0.80753821 0.80472527 0.80051768 0.80755002 0.81736576
 0.81315817 0.8201787 ]
[ 4  1 20 17 14  5  6 16  7 19 13  2 10 12 15 18 11  7  9  3]


In [23]:
print(random_search.cv_results_['params'][1])

{'criterion': 'entropy', 'max_depth': 6, 'max_features': 6, 'n_estimators': 46}


### Halving Grid search & Random search

successive halving 알고리즘을 적용하여 기존 그리드서치, 랜덤 서치에 비해 효율적으로 탐색하는 방법.
일종의 하이퍼 파라미터에 대한 토너먼트로 이해할 수 있음.

- 초기 라운드에서는 적은 데이터 혹은 자원으로 많은 하이퍼 파라미터 조합에 대해 테스트.
- 매 라운드 (또는 이터레이션)이 종료되면 결과가 좋았던 조합만 남김.
- 다음 라운드에서는 이전 라운드에 비해 더 많은 리소스를 할당 + 더 적은 조합에 대해 테스트.

## 3. Bayesian Search

베이지안 최적화는 이전의 평가 결과를 바탕으로 하이퍼파라미터의 최적 값이 어디에 있을지를 예측하고, 그 결과를 이용하여 다음 탐색 위치를 결정하는 방법입니다.

- 장점: 반복을 통해 최적의 결과를 더 빠르게 수렴할 수 있습니다.
- 단점: 상대적으로 복잡한 알고리즘.

**그리드 서치 및 랜덤 서치에 비해 효율적으로 파라미터 탐색 가능**

scikit-learn은 기본적으로 베이지안 최적화를 지원하지 않지만, Scikit-Optimize 라이브러리의 BayesSearchCV() 함수를 사용하여 베이지안 최적화를 수행할 수 있습니다.

In [24]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [26]:
from skopt import BayesSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier

# 기본 모델 설정
RF = RandomForestClassifier(random_state=0)

# 탐색하기 위한 파라미터 분포 설정
param_dist = {
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_depth": (3, 10),
    "n_estimators": (10, 100),
    "max_features": (1, 9),
}

# 베이지안 서치 인스턴스 설정 및 수행
bayesian_search = BayesSearchCV(
    estimator=RF,
    search_spaces=param_dist,
    n_iter=10,
    cv=3,
    random_state=0
)

# 베이지안 서치 실행
bayesian_search.fit(X_train, y_train)

# 결과 출력
print("Best parameters: ", bayesian_search.best_params_)
print("Best model: ", bayesian_search.best_estimator_)

Best parameters:  OrderedDict([('criterion', 'log_loss'), ('max_depth', 8), ('max_features', 4), ('n_estimators', 27)])
Best model:  RandomForestClassifier(criterion='log_loss', max_depth=8, max_features=4,
                       n_estimators=27, random_state=0)


In [27]:
print(bayesian_search.cv_results_["mean_test_score"])
print(bayesian_search.cv_results_["rank_test_score"])

[0.82299164 0.80612583 0.80612583 0.81035705 0.81598884 0.82297982
 0.82437448 0.81737758 0.81455873 0.81879587]
[2 9 9 8 6 3 1 5 7 4]


In [37]:
# bayesian CV로 얻은 best_estimator에 대한 평가
print("Accuracy: ", accuracy_score(y_test, bayesian_search.predict(X_test)))

Accuracy:  0.8156424581005587


#  3. 회귀 문제 예시

다음은 sklearn 라이브러리에서 제공하는 toy dataset 중 하나인 diabetes 데이터를 통해 regression을 수행해보도록하겠습니다.

In [28]:
from sklearn.datasets import load_diabetes

dia_x, dia_y = load_diabetes(return_X_y=True, as_frame=True)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(dia_x, dia_y, test_size = 0.2, random_state = 0)

In [32]:
from sklearn.ensemble import RandomForestRegressor

RF_reg = RandomForestRegressor(random_state=0)
RF_reg.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [33]:
y_hat = RF_reg.predict(X_test)

In [34]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('MAE : ',mean_absolute_error(y_true = y_test, y_pred = y_hat))
print('MAPE : ',mean_absolute_percentage_error(y_true = y_test, y_pred = y_hat))
print('R2 : ',r2_score(y_true = y_test, y_pred = y_hat))


MAE :  47.75224719101124
MAPE :  0.3927563646069564
R2 :  0.26865181564422547
